In [35]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [36]:
from fosforml.model_manager.snowflakesession import get_session
session = get_session()
session.get_current_schema().replace('"','')

'INSIGHT_SPCS_SCHEMA'

In [188]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake
def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [342]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv")

df_filtered = df[df['SALARY']!='low']
snow_df = session.create_dataframe(df_filtered)
# snow_df = session.create_dataframe(df)


snow_df = snow_df.drop(['Unnamed: 0','DEPARTMENT'])
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
label_encoder_column = []     
input_data_frame = apply_label_encoding(input_data_frame)
test_df = apply_label_encoding(test_df)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']

In [343]:
input_data_frame.limit(2).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |0.11                  |0.88               |7                 |272                     |4                     |0                |1       |0                        |
------------------------------------------------------------------------------------------------------------------------

In [344]:
pipeline = RandomForestClassifier(input_cols=feature_cols, label_cols=target_col)
pipeline.fit(input_data_frame)
pred_snow_df = pipeline.predict(test_df)
prob_snow_df = pipeline.predict_proba(test_df)
print(prob_snow_df.columns)

The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


['SALARY', 'SATISFACTION_LEVEL', 'LAST_EVALUATION', 'NUMBER_PROJECT', 'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY', 'WORK_ACCIDENT', 'LEFT', 'PROMOTION_LAST_5YEARS', '"""PREDICT_PROBA_0.0"""', '"""PREDICT_PROBA_1.0"""']


In [345]:
# pred_snow_df.columns

In [346]:
prob_snow_df_pandas = prob_snow_df.to_pandas()
prob_snow_df_pandas['OUTPUT_SALARY'] = pred_snow_df.to_pandas()['OUTPUT_SALARY']
final_df = session.create_dataframe(prob_snow_df_pandas)

In [347]:
final_df.columns

['SALARY',
 'SATISFACTION_LEVEL',
 'LAST_EVALUATION',
 'NUMBER_PROJECT',
 'AVERAGE_MONTLY_HOURS',
 'TIME_SPEND_COMPANY',
 'WORK_ACCIDENT',
 'LEFT',
 'PROMOTION_LAST_5YEARS',
 '"""PREDICT_PROBA_0.0"""',
 '"""PREDICT_PROBA_1.0"""',
 'OUTPUT_SALARY']

In [348]:
feature_columns = input_data_frame.columns
sf_current = final_df
sf_reference = final_df
target_column  = "SALARY"
prediction_column = "OUTPUT_SALARY"
# predict_probability_columns = [ '"""PREDICT_PROBA_high"""','"""PREDICT_PROBA_medium"""']
predict_probability_columns = ['"""PREDICT_PROBA_0.0"""',
 '"""PREDICT_PROBA_2.0"""']
drift_configs = {
        "project_id" : "asvw4g124-124sdvrgwef",
        "feature_columns": feature_columns,
        "current_dataset" : sf_current,
        "reference_dataset" : sf_reference,
        "model_name" : "HR_CHURN_MODEL",
        "problem_type" : "classification",  # classification or regression
        "target_column" : "SALARY",
        "prediction_column" : prediction_column,
        "predict_probability_columns" : predict_probability_columns,
        "version_name" : "V2",
        "alerts_configurations" : [
            {
                "parameter": "performance_drift_score",
                "threshold_range": [[0.5, 1,'Red'],[0.5, 1,'Amber'],[0.5, 1,'Green']]
            },
            {
                "parameter": "drift_score",
                "threshold_range": [[0.5, 1,'Red'],[0.5, 1,'Amber'],[0.5, 1,'Green']]
            }
        ]
    }

In [ ]:
import pandas as pd
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, try_cast, when, cast
from sklearn.datasets import load_diabetes
import json
from snowflake.ml.modeling.metrics import (confusion_matrix,
                                                accuracy_score,
                                                f1_score, recall_score,
                                                precision_score,
                                                log_loss ,
                                                roc_auc_score ,
                                            )

from snowflake.ml.modeling.metrics import explained_variance_score, \
                    mean_absolute_percentage_error ,\
					mean_squared_error, \
					mean_absolute_error, \
					r2_score

In [349]:
def is_numeric_col(df,col_name):
    status = True
    try:
        df_numeric_col_count = df.with_column("converted_column",try_cast(col(col_name),"INTEGER")) \
                                                    .filter(col("converted_column").is_not_null()).count()
        print(df_numeric_col_count,'try_cast')
        if not df_numeric_col_count == df.count():
            status = False
    except :
        pass

    try:
        df_numeric_col_count = df.with_column("converted_column",cast(col(col_name),"INTEGER")) \
                                                    .filter(col("converted_column").is_not_null()).count()
        print(df_numeric_col_count,'cast')
        if not df_numeric_col_count == df.count():
            print(df_numeric_col_count)
            status = False
    except :
        pass

    if not status :
        print("Its categorical col")
    else:
        print("Numerical col")

    return status

In [350]:
df = sf_current
col_name = "SALARY"
is_numeric_col(df,col_name)

1989 cast
Numerical col


True

In [351]:
execute_classification_drift(drift_configs,
                                 sf_current,
                                 sf_reference)

1989 cast
Numerical col
1989 cast
Numerical col


Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might no

{'accuracy_score': 0.828557, 'f1_score': 0.9048283561261513, 'recall_score': 0.9501758499413834, 'precision_score': 0.8636121470431539, 'log_loss': 6.179429766560558, 'roc_auc_score': 0.5227911051827059}


The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_multilabel_confusion_matrix_computer.<locals>.MultilabelConfusionMatrixComputer'>. Proceeding without creating optional arguments
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn<1.4'. Your UDF might not work when the package version is different between the server and your local environment.
Got error object of type 'NoneType' has no len() when trying to read default values from function: <function roc_auc_score.<locals>.roc_auc_score_anon_sproc at 0x7f735cecfb80>. Proceeding without crea

{'accuracy_score': 0.828557, 'f1_score': 0.9048283561261513, 'recall_score': 0.9501758499413834, 'precision_score': 0.8636121470431539, 'log_loss': 6.179429766560558, 'roc_auc_score': 0.5227911051827059}


Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_confusion_matrix_computer.<locals>.ConfusionMatrixComputer'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.metrics_utils.register_accumulator_udtf.<locals>.Accumulator'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.classification._register_confusion_matrix_computer.<locals>.ConfusionMatrixComputer'>. Proceeding without creating optional arguments
Got error object of type 'NoneType' has no len() when trying to read default values from function: <class 'snowflake.ml.modeling.metrics.metrics_utils.register_accumulator_udtf.<locals>.Accumulator'>. Proceedin

{'metrics': [{'classification_quality_metrics': {'current': {'accuracy_score': 0.828557,
     'f1_score': 0.9048283561261513,
     'recall_score': 0.9501758499413834,
     'precision_score': 0.8636121470431539,
     'log_loss': 6.179429766560558,
     'roc_auc_score': 0.5227911051827059,
     'gini': 0.046},
    'reference': {'accuracy_score': 0.828557,
     'f1_score': 0.9048283561261513,
     'recall_score': 0.9501758499413834,
     'precision_score': 0.8636121470431539,
     'log_loss': 6.179429766560558,
     'roc_auc_score': 0.5227911051827059,
     'gini': 0.046}}},
  {'class_representation': {'current': {'SALARY': {'"""PREDICT_PROBA_2.0"""': 1706,
      '"""PREDICT_PROBA_0.0"""': 283},
     'OUTPUT_SALARY': {'"""PREDICT_PROBA_2.0"""': 1877,
      '"""PREDICT_PROBA_0.0"""': 112}},
    'reference': {'SALARY': {'"""PREDICT_PROBA_2.0"""': 1706,
      '"""PREDICT_PROBA_0.0"""': 283},
     'OUTPUT_SALARY': {'"""PREDICT_PROBA_2.0"""': 1877,
      '"""PREDICT_PROBA_0.0"""': 112}}}},
  {

In [61]:
#### With Encoding ######

In [43]:
### Performance #####

In [ ]:

def get_classification_metrics(sf_df,true_cn,pred_cn,drift_configs):
    quality_metrics = {}
    no_classes = len(drift_configs['predict_probability_columns'])
    average='weighted' if no_classes > 2 else 'binary'
    quality_metrics['accuracy_score'] = accuracy_score(df=sf_df,y_true_col_names=true_cn, y_pred_col_names=pred_cn)
    quality_metrics['f1_score'] = f1_score(df= sf_df,y_true_col_names=true_cn, y_pred_col_names=pred_cn,average=average)
    quality_metrics['recall_score'] = recall_score(df=sf_df,y_true_col_names=true_cn, 
                                                y_pred_col_names=pred_cn,average=average)
    quality_metrics['precision_score'] = precision_score(df=sf_df,
                                                        y_true_col_names=true_cn,
                                                        y_pred_col_names=pred_cn,
                                                        average=average)
    quality_metrics['log_loss'] = log_loss(df=sf_df,y_true_col_names=true_cn, y_pred_col_names=pred_cn)
    if no_classes > 2 :
        roc_list = roc_auc_score(df=sf_df, 
                                                        y_true_col_names=true_cn,
                                                        y_score_col_names=drift_configs['predict_probability_columns'], 
                                                        multi_class='ovr',average=None).tolist()
        
        quality_metrics['roc_auc_score'] = sum(roc_list) / len(roc_list)
        
    elif no_classes == 2 :
        quality_metrics['roc_auc_score'] = roc_auc_score(df=sf_df, 
                                                        y_true_col_names=true_cn, 
                                                        y_score_col_names=pred_cn ,average="macro").tolist()
    else:
        quality_metrics['roc_auc_score'] = None

    print(quality_metrics)
    quality_metrics['gini'] = round((2*quality_metrics['roc_auc_score'])-1, 3) if quality_metrics['roc_auc_score'] else None

    return quality_metrics

def get_class_representation(df,target_column,prediction_column,drift_configs):
    class_count_dict = {}
    mapping_data = {index:col for index,col in enumerate(drift_configs['predict_probability_columns'])}
    temp_target_df  = df.group_by(col(target_column)).count()
    class_count_dict[drift_configs['target_column']] = {mapping_data[int(row[target_column])]:row['COUNT'] for row in temp_target_df.collect()}
    temp_prediction_df  = df.group_by(col(prediction_column)).count()
    class_count_dict[drift_configs['prediction_column']] = {mapping_data[int(row[prediction_column])]:row['COUNT'] for row in temp_prediction_df.collect()}
    return class_count_dict



def execute_classification_drift(drift_configs,
                                 sf_current,
                                 sf_reference):
    
    #################### Classification Drift Metrics #############################
    classification_quality_metrics = {}
    
    if not is_numeric_col(sf_current,drift_configs['target_column']):
        mapped_target_col_name = drift_configs['target_column'] + '_NUMERIC'
        sf_current,mapped_target_col_name_cur = map_to_numeric_col(sf_current,
                                        drift_configs['target_column'],
                                        drift_configs['predict_probability_columns'],
                                        mapped_target_col_name)
        sf_reference,mapped_target_col_name_ref = map_to_numeric_col(sf_reference,
                                        drift_configs['target_column'],
                                        drift_configs['predict_probability_columns'],
                                        mapped_target_col_name)
        drift_configs['categorical_target_col'] = drift_configs['target_column'] 
        drift_configs['numeric_target_col'] = mapped_target_col_name_ref
    else:
        drift_configs['numeric_target_col'] = drift_configs['target_column']


    if not is_numeric_col(sf_current,drift_configs['prediction_column']):
        mapped_prediction_col_name = drift_configs['prediction_column'] + '_NUMERIC'
        sf_current,mapped_target_col_name_cur = map_to_numeric_col(sf_current,
                                        drift_configs['prediction_column'],
                                        drift_configs['predict_probability_columns'],
                                        mapped_prediction_col_name)
        sf_reference,mapped_target_col_name_ref = map_to_numeric_col(sf_reference,
                                        drift_configs['prediction_column'],
                                        drift_configs['predict_probability_columns'],
                                        mapped_prediction_col_name)
        drift_configs['categorical_prediction_col'] = drift_configs['prediction_column']
        drift_configs['numeric_prediction_col'] = mapped_target_col_name_ref
    else:
        drift_configs['numeric_prediction_col'] = drift_configs['prediction_column']

        
    metrics = []
    numeric_target_col = drift_configs['numeric_target_col']
    numeric_prediction_col = drift_configs['numeric_prediction_col']

    ## Performance metrics
    classification_quality_metrics['current'] = get_classification_metrics(sf_current,numeric_target_col,numeric_prediction_col,drift_configs)
    classification_quality_metrics['reference'] = get_classification_metrics(sf_reference,numeric_target_col,numeric_prediction_col,drift_configs)
    metrics.append({
        "classification_quality_metrics" : classification_quality_metrics
    })


    ## Class Representation
    class_representation = {}
    class_representation['current'] = get_class_representation(sf_current,numeric_target_col,numeric_prediction_col,drift_configs)
    class_representation['reference'] = get_class_representation(sf_reference,numeric_target_col,numeric_prediction_col,drift_configs)
    metrics.append({
        "class_representation" : class_representation
    })


    ## Confusion Matrix
    current_confusion_matrix = confusion_matrix(df=sf_current, y_true_col_name=numeric_target_col, y_pred_col_name=numeric_prediction_col).tolist()
    reference_confusion_matrix = confusion_matrix(df=sf_reference, y_true_col_name=numeric_target_col, y_pred_col_name=numeric_prediction_col).tolist()
    metrics.append({
        "confusion_matrix" : {'current':current_confusion_matrix,'reference':reference_confusion_matrix}
    })

    ## Quality Metrics by class
    quality_metrics_by_class = {}
    current_class_metrics = {} ; reference_class_metrics = {}
    
    for i,class_name in enumerate(drift_configs['predict_probability_columns']):
        cu_class_metrics = {} 
        cu_tp = current_confusion_matrix[i][i]
        cu_fp = sum([row[i] for row in current_confusion_matrix])-cu_tp
        cu_fn = sum(current_confusion_matrix[i])-cu_tp
        cu_tn = sf_current.count() - cu_tp - cu_fp - cu_fn
        cu_class_metrics['precision_score'] = cu_tp/(cu_tp+cu_fp) if (cu_tp+cu_fp) > 0 else 0
        cu_class_metrics['recall_score'] = cu_tp/(cu_tp+cu_fn) if (cu_tp+cu_fn) > 0 else 0
        cu_class_metrics['f1_score'] = 2*(cu_class_metrics['precision_score']*cu_class_metrics['recall_score'])/(cu_class_metrics['precision_score']+cu_class_metrics['recall_score']) if (cu_class_metrics['precision_score']+cu_class_metrics['recall_score']) > 0 else 0
        cu_class_metrics['accuracy_score'] = (cu_tp+cu_tn)/(cu_tp+cu_fp+cu_fn+cu_tn)
        current_class_metrics[class_name] = cu_class_metrics

        re_class_metrics = {}
        re_tp = reference_confusion_matrix[i][i]
        re_fp = sum([row[i] for row in reference_confusion_matrix])-re_tp
        re_fn = sum(reference_confusion_matrix[i])-re_tp
        re_tn = sf_reference.count() - re_tp - re_fp - re_fn
        re_class_metrics['precision_score'] = re_tp/(re_tp+re_fp) if (re_tp+re_fp) > 0 else 0
        re_class_metrics['recall_score'] = re_tp/(re_tp+re_fn) if (re_tp+re_fn) > 0 else 0
        re_class_metrics['f1_score'] = 2*(re_class_metrics['precision_score']*re_class_metrics['recall_score'])/(re_class_metrics['precision_score']+re_class_metrics['recall_score']) if (re_class_metrics['precision_score']+re_class_metrics['recall_score']) > 0 else 0
        re_class_metrics['accuracy_score'] = (re_tp+re_tn)/(re_tp+re_fp+re_fn+re_tn)
        reference_class_metrics[class_name] = re_class_metrics
    
    quality_metrics_by_class['current'] = current_class_metrics
    quality_metrics_by_class['reference'] = reference_class_metrics
    metrics.append({
        "quality_metrics_by_class" : quality_metrics_by_class}
        )

    return {
        "metrics" : metrics
    }


In [200]:
[row['SALARY'] for row  in input_data_frame.select(col('SALARY')).distinct().collect()]

['low', 'medium', 'high']

In [201]:
[col_name.replace('PREDICT_PROBA_', '').replace("'","").replace('"','') for col_name in column_names if 'PREDICT_PROBA_' in col_name]

['0.0', '1.0', '2.0']

In [42]:
from snowflake.snowpark.functions import col, try_cast, when, lit
def map_to_numeric_col(df, col_name, columns_list, mapped_col_name):
    columns_map = {col: index for index, col in enumerate(columns_list)}
    colum_categories = [row[col_name] for row in df.select(col(col_name)).distinct().collect()]
    
    if set(colum_categories) == set(columns_map.keys()):
        print('matching columns')
        mapped_col = col(col_name)
        for category, value in columns_map.items():
            mapped_col = when(col(col_name) == category, lit(value).cast('FLOAT')).otherwise(mapped_col)
        df = df.with_column(mapped_col_name, mapped_col)
    else:
        temp_cols = [col_name.replace('PREDICT_PROBA_', '').replace("'", "").replace('"', '') for col_name in columns_list if 'PREDICT_PROBA_' in col_name]
        if set(colum_categories) == set(temp_cols):
            print('additional tasks')
            print(temp_cols)
            mapped_col = col(col_name)
            # df.show()
            for category, value in columns_map.items():
                mapped_col = when(col(col_name) == category, lit(value).cast('FLOAT')).otherwise(mapped_col)
                print(mapped_col,dir(mapped_col))
            df = df.with_column(mapped_col_name, mapped_col).collect()
            df.show()
        else:
            raise Exception("Predict probability column names are not matching with the values of target/prediction columns in the dataset")
        
    return df

In [254]:
input_df = map_to_numeric_col(df,col_name,columns_list,mapped_col_name)

In [264]:
input_df.select(input_df[mapped_col_name])

In [170]:
test_df = prob_snow_df.to_pandas()

In [171]:
test_df.head()

,SALARY,DEPARTMENT,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,"""PREDICT_PROBA_0.0""","""PREDICT_PROBA_1.0""","""PREDICT_PROBA_2.0"""
0,1.0,7.0,0.10,0.77,6,247,4,0,1,0,0.001736,0.768049,0.230215
1,1.0,8.0,0.43,0.47,2,147,3,0,1,0,0.005127,0.585659,0.409214
2,1.0,2.0,0.39,0.56,2,142,3,0,1,0,0.007157,0.449269,0.543574
3,2.0,7.0,0.36,0.52,2,147,3,0,1,0,0.013509,0.411396,0.575095
4,1.0,8.0,0.70,0.89,3,183,5,0,1,0,0.005062,0.879325,0.115613


In [18]:
test_df['SALARY'].unique()

array([1., 2., 0.])

In [19]:
test_df.columns

Index(['SALARY', 'DEPARTMENT', 'SATISFACTION_LEVEL', 'LAST_EVALUATION',
       'NUMBER_PROJECT', 'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY',
       'WORK_ACCIDENT', 'LEFT', 'PROMOTION_LAST_5YEARS', '"PREDICT_PROBA_0.0"',
       '"PREDICT_PROBA_1.0"', '"PREDICT_PROBA_2.0"'],
      dtype='object')

In [21]:
# dir(pipeline)

In [53]:
df = df.drop(['Unnamed: 0','DEPARTMENT'],axis=1).dropna()

In [61]:
from snowflake.ensemble._forest import RandomForestClassifier
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'snowflake.ensemble'

In [55]:
model = RandomForestClassifier()
x_train,x_test,y_train,y_test = train_test_split(df.drop('SALARY',axis=1),df['SALARY'])

In [56]:
x_train.columns

Index(['SATISFACTION_LEVEL', 'LAST_EVALUATION', 'NUMBER_PROJECT',
       'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY', 'WORK_ACCIDENT', 'LEFT',
       'PROMOTION_LAST_5YEARS'],
      dtype='object')

In [57]:
x_train.head()

,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS
2692,0.68,0.51,4,157,3,0,0,0
13580,0.86,0.84,3,177,3,0,0,0
7448,0.55,0.98,4,137,2,0,0,0
3409,0.58,0.86,5,206,3,0,0,0
570,0.44,0.52,2,137,3,0,1,0


In [58]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [60]:
# model.predict_proba(x_test)

In [46]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="Testmode02092024",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="Notebook",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_TESTMODE02092024' registered successfully."

In [54]:
model_obj = model_registry.get_model(model_name="TestModel")

In [55]:
model = model_obj.version(version_name="v1")

In [58]:
inptu_df ,test_df = session.table("HR_CHURN").randomSplit([0.75, 0.25])

In [59]:
test_df = apply_label_encoding(test_df)

In [75]:
test_df_pandas = test_df.to_pandas()

In [76]:
session.write_pandas(test_df_pandas,table_name="TEST_DATA_HR",overwrite=True).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [80]:
test_df_fetch = session.table("TEST_DATA_HR")

In [81]:
test_df_fetch.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [82]:
model.run(test_df_fetch, function_name="predict").to_pandas()

,SALARY,DEPARTMENT,Unnamed: 0,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,OUTPUT_SALARY
0,1.0,3.0,34,0.84,0.87,4,246,6,0,1,0,1.0
1,2.0,6.0,66,0.43,0.54,2,153,3,0,1,0,2.0
2,1.0,7.0,94,0.09,0.83,6,255,4,0,1,0,1.0
3,1.0,8.0,130,0.10,0.83,6,292,4,0,1,0,1.0
4,1.0,7.0,163,0.79,0.89,5,239,5,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7447,2.0,5.0,14821,0.09,0.95,7,256,4,0,1,0,2.0
7448,2.0,9.0,14859,0.45,0.54,2,129,3,0,1,0,2.0
7449,1.0,7.0,14888,0.39,0.49,2,142,3,0,1,0,1.0
7450,2.0,7.0,14945,0.14,0.75,4,277,5,1,1,0,1.0


In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)